## Result to JSON

In [8]:
import json 
import os

def json2conll(inputfile):
    outfile = inputfile.replace("json","conll")
    
    with open(inputfile) as json_file:
        data = json.load(json_file)
    mentions = data['mentions']
    cluster2id = {clustId: str(index) for index, clustId in enumerate(set([m["clustId"] for m in mentions]))}

    currentMention = mentions.pop(0)
    topic = inputfile.split('_', 1)[1].replace(".json","")
    conll = [['#begin document ' + topic]]
    for token in data["tokens"]:

        cluster = "-"
        if token["i"] == currentMention['start']: 
            cluster = "({}".format(cluster2id[currentMention["clustId"]])
        if token["i"] == currentMention['end']: 
            if currentMention['start'] == currentMention['end']:
                cluster += ")"
            else:
                cluster = "{})".format(cluster2id[currentMention["clustId"]])
            if mentions:
                currentMention = mentions.pop(0)
        conll.append([token["doc_name"], str(token["i"]), token["text"], cluster ])

    conll.append(["#end"])
    with open(outfile, 'w') as conll_file:
        for row in conll:
            conll_file.write("\t".join(row) + "\n")

## Process Review HIT

In [10]:
import json 
import pandas as pd
import html
from pathlib import Path
        
infiles = []
configs = []
outfile = ""
for inputfile in infiles:
    with open(inputfile) as json_file:
        data = json.load(json_file)
        
    config = {
        "mode":"reviewer",
        "topic":Path(inputfile).stem,
        "tokens":data["tokens"],
        "selectedCluster": data['mentions'][0]["clustId"],
        "viewedMentions": [data['mentions'][0]],
        "currentMention": data['mentions'][1],
        "candidateMentions": data['mentions'][2:]
    }
    config_string = html.escape(json.dumps(config))
    configs.append({"data":config_string})

df = pd.DataFrame(configs).to_csv(outfile, index=False)


In [9]:

directory = r'C:\Users\abornst\Documents\thesis\data\reviewer_trial'
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        json2conll(os.path.join(directory, filename))
    else:
        continue
